***
**Importing necessary libraries**
***

In [ ]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm

***
**Merging Google Drive**
***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***
**Dataset Path**
***

In [ ]:
dataset='/content/drive/MyDrive/BRAC /Past Courses/Thesis/Thesis Final/Dataset/CNN'

***
**Installing and importing mne for reading EEG data**
***

In [ ]:
!pip install mne

     |████████████████████████████████| 6.9 MB 4.9 MB/s 


In [ ]:
import mne

***
**Creating Classes**
***


In [ ]:
class_names = ['Healthy', 'MDD']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

***
**Data Preprocessing**
***

In [ ]:
images = []
labels = []
output=[]
IMAGE_SIZE = (150, 150)
print("Loading {}".format(dataset))

# Iterate through each folder corresponding to a category
for folder in os.listdir(dataset):
    label = class_names_label[folder]
    
    # Iterate through each image in our folder
    for file in tqdm(os.listdir(os.path.join(dataset, folder))):
        
        # Get the path name of the image
        img_path = os.path.join(os.path.join(dataset, folder), file)

        # Open and resize the img
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, IMAGE_SIZE) 
        
        # Append the image and its corresponding label to the output
        images.append(image)
        labels.append(label)

images = np.array(images, dtype = 'float32')
labels = np.array(labels, dtype = 'int32')

output.append((images, labels))

Loading /content/drive/MyDrive/BRAC /Past Courses/Thesis/Pre Thesis II/Final Paper of Pre Thesis II/14- Used Dataset/Sorted Data/Image Version


100%|██████████| 170/170 [00:35<00:00,  4.79it/s]


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

***
**Creating the 2D CNN model**
***

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

***
**Accuracy Level**
***

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

***
**Data Training**
***

In [ ]:
history = model.fit(X_test, y_test, batch_size=5, epochs=20)

Epoch 1/20
15/15 [==============================] - 3s 226ms/step - loss: 2.8502 - accuracy: 0.5972
Epoch 2/20
15/15 [==============================] - 3s 209ms/step - loss: 0.9168 - accuracy: 0.7639
Epoch 3/20
15/15 [==============================] - 3s 209ms/step - loss: 0.5595 - accuracy: 0.7361
Epoch 4/20
15/15 [==============================] - 3s 205ms/step - loss: 0.5549 - accuracy: 0.7778
Epoch 5/20
15/15 [==============================] - 3s 209ms/step - loss: 0.5424 - accuracy: 0.7639
Epoch 6/20
15/15 [==============================] - 3s 207ms/step - loss: 0.5684 - accuracy: 0.6944
Epoch 7/20
15/15 [==============================] - 3s 216ms/step - loss: 0.5708 - accuracy: 0.7222
Epoch 8/20
15/15 [==============================] - 3s 212ms/step - loss: 0.6344 - accuracy: 0.7222
Epoch 9/20
15/15 [==============================] - 3s 209ms/step - loss: 0.4968 - accuracy: 0.7500
Epoch 10/20
15/15 [==============================] - 3s 214ms/step - loss: 0.4624 - accuracy: 0.8333

***
**Prediction**
***

In [ ]:
predictions = model.predict(X_test)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability